In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model

In [2]:
dados_clientes = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', sep = ',', header = None)
dados_clientes.head(249)
dados_clientes = dados_clientes.rename(columns = {0:'Homem', 1:'Idade', 2:'Dívida', 3:'Casado', 4:'ClienteBanco', 5:'Escolaridade', 6:'Etnia', 7:'AnosEmpregado', 8:'Precedente', 9:'Empregado', 10:'Escore', 11:'CarteiraMotorista', 12: 'Cidadania', 13:'CEP', 14:'Remuneracao', 15:'Aprovado'})




Olhando o dataset no detalhe podemos identificar que os valores foram convertidos para caracteres numéricos e não numéricos, aparentemente sem significado, por motivos de confidencialidade, porém, em pesquisa realizada na internet é possível induzir do que se trata cada coluna pelo tipo de informação contida nela. As colunas são as que seguem:

In [3]:
def_col = {0:'Homem', 1:'Idade', 2:'Dívida', 3:'Casado', 4:'ClienteBanco', 5:'Escolaridade', 6:'Etnia', 7:'AnosEmpregado', 8:'Precedente', 9:'Empregado', 10:'Escore', 11:'CarteiraMotorista', 12: 'Cidadania', 13:'CEP', 14:'Remuneracao', 15:'Aprovado' }
print(def_col)

{0: 'Homem', 1: 'Idade', 2: 'Dívida', 3: 'Casado', 4: 'ClienteBanco', 5: 'Escolaridade', 6: 'Etnia', 7: 'AnosEmpregado', 8: 'Precedente', 9: 'Empregado', 10: 'Escore', 11: 'CarteiraMotorista', 12: 'Cidadania', 13: 'CEP', 14: 'Remuneracao', 15: 'Aprovado'}


Descrevendo melhor os dados, percebemos que apenas as colunas 2, 7, 10 e 14 são númericas. Estes campos são elegíveis para a construção dos modelos porém não podemos simplesmente ignorar o restante das colunas não-numéricas e faltantes, sob o risco de impactarmos na performance dos modelos, caso alguma variável associada seja perdida ao longo do processo e por isso precisam ser tratados.

In [4]:
dados_clientes_descricao = dados_clientes.describe()
print (dados_clientes_descricao)
dados_clientes_info = dados_clientes.info()
print(dados_clientes_info)
dados_clientes.tail(17)

           Dívida  AnosEmpregado     Escore    Remuneracao
count  690.000000     690.000000  690.00000     690.000000
mean     4.758725       2.223406    2.40000    1017.385507
std      4.978163       3.346513    4.86294    5210.102598
min      0.000000       0.000000    0.00000       0.000000
25%      1.000000       0.165000    0.00000       0.000000
50%      2.750000       1.000000    0.00000       5.000000
75%      7.207500       2.625000    3.00000     395.500000
max     28.000000      28.500000   67.00000  100000.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Homem              690 non-null    object 
 1   Idade              690 non-null    object 
 2   Dívida             690 non-null    float64
 3   Casado             690 non-null    object 
 4   ClienteBanco       690 non-null    object 
 5   Escolaridade       690 

,Homem,Idade,Dívida,Casado,ClienteBanco,Escolaridade,Etnia,AnosEmpregado,Precedente,Empregado,Escore,CarteiraMotorista,Cidadania,CEP,Remuneracao,Aprovado
673,?,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


É possível dizer também nesta primeira análise que o dataset possui campos de vários intervalos. O campo 'remuneração' por exemplo, vai de 0 até 100000.  É possível identificar também que os campos com '?' tratam-se de valores faltantes e devemos transformá-los para a variável NaN (Not a Number) para que as operações identifiquem como um valor faltante. Este passo é feito a seguir:

In [5]:
print(dados_clientes.isnull().values.sum())
dados_clientes = dados_clientes.replace('?', np.nan)
print('Total NaN: ' + str(dados_clientes.isnull().values.sum()))
print('NaN by column:' '\n')
print(dados_clientes.isnull().sum())
dados_clientes.tail(17)

0
Total NaN: 67
NaN by column:

Homem                12
Idade                12
Dívida                0
Casado                6
ClienteBanco          6
Escolaridade          9
Etnia                 9
AnosEmpregado         0
Precedente            0
Empregado             0
Escore                0
CarteiraMotorista     0
Cidadania             0
CEP                  13
Remuneracao           0
Aprovado              0
dtype: int64


,Homem,Idade,Dívida,Casado,ClienteBanco,Escolaridade,Etnia,AnosEmpregado,Precedente,Empregado,Escore,CarteiraMotorista,Cidadania,CEP,Remuneracao,Aprovado
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


In [6]:
dados_clientes.fillna(dados_clientes.mean(), inplace=True)
print('Total NaN: ' + str(dados_clientes.isnull().values.sum()))
dados_clientes.isnull().sum()


Total NaN: 67


Homem                12
Idade                12
Dívida                0
Casado                6
ClienteBanco          6
Escolaridade          9
Etnia                 9
AnosEmpregado         0
Precedente            0
Empregado             0
Escore                0
CarteiraMotorista     0
Cidadania             0
CEP                  13
Remuneracao           0
Aprovado              0
dtype: int64

A seguir, por se tratar de dados categóricos, inputamos os valores mais frequentes para as colunas onde temos dados não numéricos.

In [7]:
for col in dados_clientes:    
    if dados_clientes[col].dtypes == 'object':        
        dados_clientes = dados_clientes.fillna(dados_clientes[col].value_counts().index[0])      
        
print('Total missing values:' + str(dados_clientes.isnull().values.sum()))
print('Missing values in each column:')
dados_clientes.isnull().sum()

Total missing values:0
Missing values in each column:


Homem                0
Idade                0
Dívida               0
Casado               0
ClienteBanco         0
Escolaridade         0
Etnia                0
AnosEmpregado        0
Precedente           0
Empregado            0
Escore               0
CarteiraMotorista    0
Cidadania            0
CEP                  0
Remuneracao          0
Aprovado             0
dtype: int64

Agora que os dados estão devidamento tratados, precisamos ainda tratar alguns dados remanescentes antes de construir o modelo de machine learning, começando pela conversão de dados não-númericos em numéricos usando a técnica de 'Label Encoding', em seguida separar os dados de treino e de teste e por fim normalizar os dados. Este passo é de extrema importância para a performance dos modelos de machine learning, pois a maioria deles aceita somente campos estritamente numéricos. 

In [8]:
le = LabelEncoder()
for col in dados_clientes:
    if dados_clientes[col].dtypes == 'object':
        le.fit(dados_clientes[col])
        dados_clientes[col]=le.transform(dados_clientes[col])
        dados_clientes.info()      
       

        

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Homem              690 non-null    int32  
 1   Idade              690 non-null    object 
 2   Dívida             690 non-null    float64
 3   Casado             690 non-null    object 
 4   ClienteBanco       690 non-null    object 
 5   Escolaridade       690 non-null    object 
 6   Etnia              690 non-null    object 
 7   AnosEmpregado      690 non-null    float64
 8   Precedente         690 non-null    object 
 9   Empregado          690 non-null    object 
 10  Escore             690 non-null    int64  
 11  CarteiraMotorista  690 non-null    object 
 12  Cidadania          690 non-null    object 
 13  CEP                690 non-null    object 
 14  Remuneracao        690 non-null    int64  
 15  Aprovado           690 non-null    object 
dtypes: float64(2), int32(1), i

Para os valores faltantes de idade aplica-se um método de regressão linear com base na correlação dos campos AnosEmpregado e Idade:

In [9]:
dados_corr = dados_clientes[['Idade','Dívida', 'AnosEmpregado', 'Escore', 'Remuneracao']]
dados_corr.corr(method ='pearson')



,Idade,Dívida,AnosEmpregado,Escore,Remuneracao
Idade,1.000000,0.135058,0.386076,0.160599,0.016829
Dívida,0.135058,1.000000,0.298902,0.271207,0.123121
AnosEmpregado,0.386076,0.298902,1.000000,0.322330,0.051345
Escore,0.160599,0.271207,0.322330,1.000000,0.063692
Remuneracao,0.016829,0.123121,0.051345,0.063692,1.000000


In [10]:
reg = linear_model.LinearRegression()
reg.fit(dados_corr[['AnosEmpregado']],dados_corr.Idade)

LinearRegression()

In [11]:
reg.coef_

array([11.09698296])

In [12]:
reg.intercept_

125.8558892621836

A seguir dividimos os dados em tabelas de treino e de teste. Para garantir a consistência dos resultados, o ideal é não utilizar os dados de teste para dimensionar os dados de treino ou para direcionar o processo de treinamento de um modelo de machine learning. Neste passo também faz-se necessários retirar os campos de 'carteira de motorista' e 'CEP' pois não são determinantes na aprovação ou não do cartão de crédito e assim melhorar a performance.

In [13]:
dados_clientes = dados_clientes.drop(['CarteiraMotorista','CEP'], axis=1)
print(dados_clientes.head())
dados_clientes = dados_clientes.values

X,y = dados_clientes[:,0:12], dados_clientes[:,13]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

   Homem  Idade  Dívida  Casado  ClienteBanco  Escolaridade  Etnia  \
0      1    156   0.000       2             1            13      8   
1      0    328   4.460       2             1            11      4   
2      0     89   0.500       2             1            11      4   
3      1    125   1.540       2             1            13      8   
4      1     43   5.625       2             1            13      8   

   AnosEmpregado  Precedente  Empregado  Escore  Cidadania  Remuneracao  \
0           1.25           1          1       1          0            0   
1           3.04           1          1       6          0          560   
2           1.50           1          0       0          0          824   
3           3.75           1          1       5          0            3   
4           1.71           1          0       0          2            0   

   Aprovado  
0         0  
1         0  
2         0  
3         0  
4         0  


Após a separação dos dados em treino e teste, normalizaremos os dados agora para termos uma base comparável de correlação e assim aumentar a consistência dos modelos já que os mesmos estarão em uma base de 0 a 1:

In [14]:
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

De acordo com as informações contidas na descrição dos datasets no UCI, o dataset possui mais solicitações negadas do informações aprovadas: Negadas (55.5%), Aprovadas (44.5%). Portanto o modelo escolhido precisa prever o status dessas aplicações no mesmo padrão. Por se tratar de um problema classificatório (binário) de natureza probabilística e por seu alto grau de confiabilidade, o modelo de regressão logística pode ser o ideal para este tipo de classificação.

In [15]:
logreg = LogisticRegression()
logreg.fit(rescaledX_train, y_train)

LogisticRegression()

In [16]:
y_pred = logreg.predict(rescaledX_test)

print('Accuracy of logistics regression classifier: ', logreg.score(rescaledX_test, y_test))

print('Confusion matrix: \n '), confusion_matrix(y_test, y_pred)

Accuracy of logistics regression classifier:  0.8377192982456141
Confusion matrix: 
 


(None,
 array([[93, 10],
        [27, 98]], dtype=int64))

Usa-se a pesquisa Grid a seguir para ajustar os parâmetros e avaliar metodicamente um modelo para cada combinação.
Em seguida instruímos o GridSearchCV a fazer uma validação cruzada de 5 interações.


In [17]:

from sklearn.model_selection import GridSearchCV
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]
param_grid = dict(tol= tol, max_iter= max_iter)


In [18]:

grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
rescaledX = scaler.fit_transform(X)
grid_model_result = grid_model.fit(rescaledX, y)
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.850725 using {'max_iter': 100, 'tol': 0.01}


Após a validação cruzada, a acurácia do modelo se manteve em 85%. O que nos indica que o mesmo está bem calibrado e que todo o pré-processamento da base foi feito de maneira satisfatória. 